# Data Cleanup

## Setup

### Import Packages

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from collections import OrderedDict
from datetime import datetime, date
from os import environ
import json

# Set ipython's max row display
pd.set_option('display.max_row', 1000)
# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [10, 5]

### Global Variables

## Importing Dataset

### Combined dataset

In [2]:
read_name = "../data/bronze_tables/combined_dataset.csv"

df_combined = pd.read_csv(read_name)

C:\Users\abdul\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_combined.head()

,Unnamed: 0,Entity_x,Code,Year,Annual CO2 emissions,Entity_y,Fossil fuels (% growth),Entity_x.1,Annual change in primary energy consumption (%),Entity_y.1,Gas Consumption - TWh,Coal Consumption - TWh,Oil Consumption - TWh,Entity_x.2,Fossil fuels (TWh),Entity_y.2,Coal Production - TWh,Oil Production - TWh,Gas Production - TWh,Entity_x.3,Fossil fuels per capita (kWh),Entity_y.3,Fossil fuels (% equivalent primary energy),Entity_x.4,"Gas (TWh, direct energy)","Oil (TWh, direct energy)","Coal (TWh, direct energy)",Entity_y.4,Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,Entity_x.5,prod of Electricity from wind (TWh),prod of Electricity from hydro (TWh),prod of Electricity from solar (TWh),prod of Other renewables including bioenergy (TWh),Entity_y.5,Per capita electricity (kWh),Entity_x.6,Renewables per capita (kWh - equivalent),Entity_y.6,Renewables (% electricity)
0,0,Afghanistan,AFG,1949,14656.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Afghanistan,AFG,1950,84272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Afghanistan,AFG,1951,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Afghanistan,AFG,1952,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Afghanistan,AFG,1953,106256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_combined = df_combined[df_combined.columns[1:]]

In [5]:
df = df_combined.copy()

#### Unify Entity Columns

In [6]:
entity_columns = []
for col in df.columns:
    if "Entity" in col:
        entity_columns.append(col)

In [7]:
df_entities = df[entity_columns + ['Code']].drop_duplicates(entity_columns)

In [8]:
df['Entity'] = df['Entity_x']

for col in entity_columns:
    df['Entity'] = df['Entity'].fillna(df[col])

In [9]:
df['Entity'].isna().sum()

0

In [10]:
df = df.drop(columns=entity_columns)

In [11]:
df[['Entity', 'Year']].value_counts()

Entity       Year
Afghanistan  1949    1
Panama       1968    1
             1976    1
             1975    1
             1974    1
                    ..
Georgia      1957    1
             1956    1
             1955    1
             1954    1
Zimbabwe     2021    1
Length: 21879, dtype: int64

#### Drop Duplicate Columns

In [12]:
cols = list(df.columns)
unique_cols = {}

for col in cols:
    unique_cols[col] = col.split('_', 1)[0]

In [13]:
df = df.rename(columns = unique_cols)

In [14]:
df = df.loc[:,~df.columns.duplicated()]

ordered_columns = ['Entity', *df.columns[:-1]]
df = df[ordered_columns]

In [15]:
df.head()

,Entity,Code,Year,Annual CO2 emissions,Fossil fuels (% growth),Annual change in primary energy consumption (%),Gas Consumption - TWh,Coal Consumption - TWh,Oil Consumption - TWh,Fossil fuels (TWh),Coal Production - TWh,Oil Production - TWh,Gas Production - TWh,Fossil fuels per capita (kWh),Fossil fuels (% equivalent primary energy),"Gas (TWh, direct energy)","Oil (TWh, direct energy)","Coal (TWh, direct energy)",Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,prod of Electricity from wind (TWh),prod of Electricity from hydro (TWh),prod of Electricity from solar (TWh),prod of Other renewables including bioenergy (TWh),Per capita electricity (kWh),Renewables per capita (kWh - equivalent),Renewables (% electricity)
0,Afghanistan,AFG,1949,14656.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1950,84272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1951,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1952,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1953,106256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Income Level Lookup table

In [16]:
read_name = "../data/bronze_tables/income_level_lookup.xlsx"

df_income_lookup = pd.read_excel(read_name, skiprows=0)

df_income_lookup.head()

,Economy,Code,Region,Income group,Lending category,Other (EMU or HIPC)
0,Aruba,ABW,Latin America & Caribbean,High income,NaN,NaN
1,Afghanistan,AFG,South Asia,Low income,IDA,HIPC
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,NaN
3,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN
4,Andorra,AND,Europe & Central Asia,High income,NaN,NaN


In [17]:
df_income_lookup = df_income_lookup[df_income_lookup.columns[:5]]

#### join with combined_df

In [18]:
# df_income_lookup = df_income_lookup.rename(columns={'Economy': 'Entity'})
df = df.set_index('Code').join(df_income_lookup.set_index('Code')).reset_index()

In [19]:
df[['Entity', 'Year']].value_counts()

Entity       Year
Afghanistan  1949    1
Panama       1968    1
             1976    1
             1975    1
             1974    1
                    ..
Georgia      1957    1
             1956    1
             1955    1
             1954    1
Zimbabwe     2021    1
Length: 21879, dtype: int64

### Population & Area

In [20]:
read_name = "../data/bronze_tables/API_EN.POP.DNST_DS2_en_csv_v2_4701323.csv"

df_population = pd.read_csv(read_name, skiprows=4)

df_population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,307.966667,312.411111,314.994444,316.827778,318.650000,320.566667,322.466667,324.316667,326.300000,328.166667,330.233333,332.494444,334.644444,336.261111,336.961111,336.588889,335.366667,333.900000,333.177778,333.872222,...,484.888889,494.494444,504.811111,516.066667,527.733333,538.977778,548.577778,555.711111,560.166667,562.366667,563.122222,563.622222,564.805556,566.944444,569.805556,573.138889,576.533333,579.661111,582.583333,585.338889,588.033333,590.611111,593.144444,NaN,NaN
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,9.206929,9.444024,9.690214,9.945378,10.209576,10.482895,10.765896,11.059617,11.365371,11.684171,12.016528,12.362473,12.721761,13.094008,13.478945,13.876693,14.287579,14.711372,15.148660,15.599232,...,25.947604,26.625607,27.321141,28.035481,28.769895,29.526360,30.001659,30.801413,31.628679,32.483559,33.367205,34.279582,35.219737,36.189915,36.879528,37.898051,38.940522,40.004465,41.089451,42.195162,43.319881,44.462045,45.620592,NaN,NaN
2,Afghanistan,AFG,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,14.058547,14.337645,14.631648,14.940699,15.265041,15.600080,15.945197,16.308762,16.702347,17.131463,17.594177,18.078319,18.565480,19.031569,19.455045,19.844369,20.194247,20.454746,20.561857,20.478206,...,30.261978,30.925972,31.859861,33.127872,34.651540,36.307546,37.910996,39.333171,40.527204,41.550591,42.503842,43.534959,44.747269,46.176059,47.776671,49.475786,51.164166,52.762987,54.249311,55.649251,56.992046,58.325678,59.684990,NaN,NaN
3,Africa Western and Central,AFW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,10.877837,11.109940,11.351399,11.601453,11.859717,12.126330,12.402029,12.687791,12.984878,13.294421,13.616671,13.952068,14.302085,14.669124,15.053820,15.456176,15.876543,16.313802,16.766255,17.232596,...,28.002263,28.761516,29.540244,30.338334,31.157344,32.000505,32.872024,33.774930,34.710713,35.678698,36.677028,37.702723,38.753638,39.829118,40.929916,42.056948,43.211639,44.394917,45.607045,46.847251,48.114080,49.405535,50.720207,NaN,NaN
4,Angola,AGO,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,4.436874,4.498676,4.555554,4.600140,4.628678,4.637286,4.631780,4.629801,4.655231,4.724761,4.845784,5.012405,5.211585,5.423617,5.634069,5.839119,6.043005,6.249117,6.463553,6.690695,...,12.320205,12.727096,13.151101,13.592487,14.052633,14.535555,15.046232,15.588036,16.162593,16.768557,17.402451,18.059096,18.734457,19.427817,20.139508,20.867720,21.610470,22.366552,23.135062,23.916555,24.713072,25.527632,26.362612,NaN,NaN


In [21]:
read_name = "../data/bronze_tables/API_AG.LND.TOTL.K2_DS2_en_csv_v2_4701206.csv"

df_area = pd.read_csv(read_name, skiprows=4)

df_area.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,Land area (sq. km),AG.LND.TOTL.K2,NaN,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,...,180.0,180.0,180.0,180.0,180.0,180.0,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,180.00,1.800000e+02,1.800000e+02,180.00,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,NaN,NaN
1,Africa Eastern and Southern,AFE,Land area (sq. km),AG.LND.TOTL.K2,NaN,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,...,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,1.472019e+07,1.472024e+07,1.472023e+07,1.472027e+07,14720236.89,1.472027e+07,1.472096e+07,14721240.05,1.484517e+07,1.484513e+07,1.484509e+07,1.484514e+07,1.484515e+07,1.484514e+07,1.484515e+07,1.484516e+07,1.484512e+07,NaN,NaN
2,Afghanistan,AFG,Land area (sq. km),AG.LND.TOTL.K2,NaN,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,...,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,652230.00,6.522300e+05,6.522300e+05,652230.00,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,NaN,NaN
3,Africa Western and Central,AFW,Land area (sq. km),AG.LND.TOTL.K2,NaN,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046180.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,...,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9045780.00,9.045780e+06,9.045780e+06,9045780.00,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045773e+06,NaN,NaN
4,Angola,AGO,Land area (sq. km),AG.LND.TOTL.K2,NaN,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,...,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1246700.00,1.246700e+06,1.246700e+06,1246700.00,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,NaN,NaN


In [22]:
df_pop_area = pd.concat([df_population, df_area])

df_pop_area = df_pop_area[df_population.columns[:-2]].drop(['Indicator Code'], axis=1)

In [23]:
df_pop_area.head(2)

,Country Name,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,Population density (people per sq. km of land ...,NaN,307.966667,312.411111,314.994444,316.827778,318.650000,320.566667,322.466667,324.316667,326.300000,328.166667,330.233333,332.494444,334.644444,336.261111,336.961111,336.588889,335.366667,333.900000,333.177778,333.872222,336.450000,...,462.283333,474.722222,484.888889,494.494444,504.811111,516.066667,527.733333,538.977778,548.577778,555.711111,560.166667,562.366667,563.122222,563.622222,564.805556,566.944444,569.805556,573.138889,576.533333,579.661111,582.583333,585.338889,588.033333,590.611111,593.144444
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,NaN,9.206929,9.444024,9.690214,9.945378,10.209576,10.482895,10.765896,11.059617,11.365371,11.684171,12.016528,12.362473,12.721761,13.094008,13.478945,13.876693,14.287579,14.711372,15.148660,15.599232,16.062631,...,24.633762,25.284822,25.947604,26.625607,27.321141,28.035481,28.769895,29.526360,30.001659,30.801413,31.628679,32.483559,33.367205,34.279582,35.219737,36.189915,36.879528,37.898051,38.940522,40.004465,41.089451,42.195162,43.319881,44.462045,45.620592


In [24]:
df_pop_area = df_pop_area.melt(
    id_vars=df_pop_area.columns[:3], 
    value_vars=df_pop_area.columns[3:],
    var_name='Year'
).reset_index(drop=True)

df_pop_area.head()

,Country Name,Country Code,Indicator Name,Year,value
0,Aruba,ABW,Population density (people per sq. km of land ...,1960,NaN
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,1960,NaN
2,Afghanistan,AFG,Population density (people per sq. km of land ...,1960,NaN
3,Africa Western and Central,AFW,Population density (people per sq. km of land ...,1960,NaN
4,Angola,AGO,Population density (people per sq. km of land ...,1960,NaN


In [25]:
df_pop_area = df_pop_area.pivot(['Country Code', 'Year'], 'Indicator Name', 'value').reset_index()

In [26]:
df_pop_area['Year'] = df_pop_area['Year'].astype(np.number)

#### join with combined_df

In [27]:
df_pop_area = df_pop_area.rename(columns={'Country Name': 'Entity', 'Country Code': 'Code'})

In [28]:
df_pop_area.head()

Indicator Name,Code,Year,Land area (sq. km),Population density (people per sq. km of land area)
0,ABW,1960.0,NaN,NaN
1,ABW,1961.0,180.0,307.966667
2,ABW,1962.0,180.0,312.411111
3,ABW,1963.0,180.0,314.994444
4,ABW,1964.0,180.0,316.827778


In [29]:
df = df.set_index(['Code', 'Year']).join(df_pop_area.set_index(['Code', 'Year'])).reset_index()

In [30]:
df.query("Entity == 'Saudi Arabia' and Year == 2019")

,Code,Year,Entity,Annual CO2 emissions,Fossil fuels (% growth),Annual change in primary energy consumption (%),Gas Consumption - TWh,Coal Consumption - TWh,Oil Consumption - TWh,Fossil fuels (TWh),Coal Production - TWh,Oil Production - TWh,Gas Production - TWh,Fossil fuels per capita (kWh),Fossil fuels (% equivalent primary energy),"Gas (TWh, direct energy)","Oil (TWh, direct energy)","Coal (TWh, direct energy)",Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,prod of Electricity from wind (TWh),prod of Electricity from hydro (TWh),prod of Electricity from solar (TWh),prod of Other renewables including bioenergy (TWh),Per capita electricity (kWh),Renewables per capita (kWh - equivalent),Renewables (% electricity),Economy,Region,Income group,Lending category,Land area (sq. km),Population density (people per sq. km of land area)
17191,SAU,2019,Saudi Arabia,622412749.0,-1.391733,-1.387501,1111.5,1.172558,1884.200928,2996.873535,NaN,6472.839355,1111.5,87452.648438,99.981819,NaN,NaN,NaN,0.0,0.206737,0.0,0.0,0.0,0.0,0.206737,0.0,9362.818359,0.015902,0.061631,Saudi Arabia,Middle East & North Africa,High income,NaN,2149690.0,15.941149


In [31]:
df[['Entity', 'Year']].value_counts()

Entity       Year
Afghanistan  1949    1
Panama       1968    1
             1976    1
             1975    1
             1974    1
                    ..
Georgia      1957    1
             1956    1
             1955    1
             1954    1
Zimbabwe     2021    1
Length: 21879, dtype: int64

### Rearrange Columns

In [32]:
cat_cols = ['Entity', 'Year', 'Code', 'Region', 'Income group', 'Lending category']
value_cols = list((set(df.columns) - set(cat_cols)) - {'Country Code'})

In [33]:
df = df[cat_cols + value_cols]

In [34]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels per capita (kWh),Oil Consumption - TWh,Economy,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh)
0,Aruba,1926,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33394.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,1927,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,1928,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,1929,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103410.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,1930,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df[['Entity', 'Year']].value_counts()

Entity       Year
Afghanistan  1949    1
Panama       1968    1
             1976    1
             1975    1
             1974    1
                    ..
Georgia      1957    1
             1956    1
             1955    1
             1954    1
Zimbabwe     2021    1
Length: 21879, dtype: int64

### Basic Analysis

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21879 entries, 0 to 21878
Data columns (total 35 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               21879 non-null  object 
 1   Year                                                 21879 non-null  int64  
 2   Code                                                 21879 non-null  object 
 3   Region                                               20587 non-null  object 
 4   Income group                                         20476 non-null  object 
 5   Lending category                                     12829 non-null  object 
 6   Fossil fuels per capita (kWh)                        4270 non-null   float64
 7   Oil Consumption - TWh                                4299 non-null   float64
 8   Economy                                              20587 non-nul

In [37]:
df.describe()

,Year,Fossil fuels per capita (kWh),Oil Consumption - TWh,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh)
count,21879.000000,4270.000000,4299.000000,74.000000,4230.000000,4230.000000,4270.000000,7050.000000,1397.000000,2514.000000,74.000000,5828.000000,5665.000000,2.129900e+04,4270.000000,4292.000000,2553.000000,11042.000000,4273.000000,4292.000000,7113.000000,9354.000000,4230.000000,1.197800e+04,4296.000000,74.000000,7178.000000,4255.000000,7113.000000
mean,1960.890443,32590.368226,1015.737954,23161.696104,5.525746,6.126806,86.307820,3.366912,1915.282764,941.158999,16669.849590,3924.760158,29.633481,1.572975e+08,2346.057603,62.145414,1707.803287,270.028485,756.214677,4.578573,1.344705,4.533664,2.250895,6.467467e+05,564.805312,30469.016364,38.012262,3.538322,3.658208
std,47.719532,34472.620328,4706.367669,13521.988256,34.808495,61.993965,15.585076,27.092075,6167.601614,3615.938174,12227.579351,5117.133819,32.033628,1.381852e+09,11082.105692,308.745114,5858.493747,1308.026452,3785.186997,13.474483,22.090646,31.757811,28.611929,1.812242e+06,2785.492795,17904.778196,240.564170,26.995583,47.898591
min,1750.000000,155.807388,0.097381,97.000000,0.000000,0.000000,12.804697,0.000000,0.339596,0.000000,0.000000,0.000000,0.000000,3.400000e+01,0.097381,0.000000,0.000000,0.098625,0.000000,0.000000,0.000000,-95.005081,0.000000,1.000000e+01,0.000000,0.000000,0.000000,-49.590828,0.000000
25%,1937.000000,11678.774170,65.349743,16180.420654,0.000000,0.000000,80.717112,0.000000,37.385506,45.443999,6995.105713,572.427170,1.366416,4.506720e+05,117.750929,0.430750,102.900040,19.113774,3.395960,0.172228,0.000000,-0.781557,0.000000,1.827000e+04,8.854822,19688.894043,0.017904,-1.007167,0.000000
50%,1972.000000,25363.719727,142.951569,23494.454102,0.046381,0.000000,91.689793,0.000000,125.336014,144.380302,16082.451172,2469.344605,16.124605,3.572952e+06,283.652481,4.616636,344.998383,63.247178,32.584167,0.806088,0.000000,2.361172,0.000000,1.074000e+05,53.573006,35453.982422,1.560000,2.493978,0.000000
75%,1997.000000,40195.003906,418.514847,30863.200195,1.373498,0.070833,97.799118,0.368750,815.064575,404.810089,25554.728027,5388.259033,53.125000,2.745129e+07,967.343674,23.120900,1131.237549,149.543597,169.332794,2.851986,0.002000,6.958872,0.005899,4.727100e+05,241.780823,44337.702148,9.962750,6.371659,0.006000
max,2021.000000,308704.218750,53368.628906,45161.207031,762.782654,1861.939819,100.000015,762.782654,46550.605469,40368.828125,40374.605469,56781.601560,100.000000,3.670250e+10,136131.468750,4345.990234,52181.949219,21388.600000,45161.207031,153.883406,1032.501221,1553.104980,1032.501221,1.638995e+07,40374.605469,53368.628906,4345.990234,1553.104980,1861.939819


In [38]:
df.describe(include='object')

,Entity,Code,Region,Income group,Lending category,Economy
count,21879,21879,20587,20476,12829,20587
unique,235,235,7,4,3,211
top,United Kingdom,GBR,Europe & Central Asia,High income,IBRD,United Kingdom
freq,272,272,7973,8629,7557,272


### Data Clean-up

In [39]:
x = 'Entity'

#### Unify Entinty Names

In [40]:
df[x] = df[x].str.title()

#### Create Entity Category Column

In [41]:
df[x].nunique()

235

In [42]:
df[x].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'Netherlands Antilles', 'United Arab Emirates', 'Argentina',
       'Armenia', 'American Samoa', 'Antarctica', 'Antigua And Barbuda',
       'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium',
       'Benin', 'Bonaire Sint Eustatius And Saba', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia And Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote D'Ivoire", 'Cameroon',
       'Democratic Republic Of Congo', 'Congo', 'Cook Islands',
       'Colombia', 'Comoros', 'Cape Verde', 'Costa Rica', 'Cuba',
       'Curacao', 'Christmas Island', 'Cayman Islands', 'Cyprus',
       'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt', 'Eritrea',
       'Weste

In [43]:
continents = ['Asia', 'Africa', 'North America', 'South America', 'Europe', "Antarctica"]

countries_official_names = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']
other_countries = [
       'Antigua And Barbuda', 'Bolivia',
       'Bonaire Sint Eustatius And Saba', 'Bosnia And Herzegovina',
       'British Virgin Islands', 'Brunei', "Cote D'Ivoire", 'Curacao',
       'Czechia', 'Democratic Republic Of Congo', 'Eswatini',
       'Faeroe Islands', 'French Equatorial Africa', 'French West Africa',
       'Iran',
       'Kosovo',  'Laos', 'Leeward Islands',
       'Moldova',
       'North Korea', 'North Macedonia', 'Oceania', 'Palestine',
       'Panama Canal Zone', 'Reunion', 'Russia', 'Ryukyu Islands',
       'Saint Helena', 'Saint Kitts And Nevis',
       'Saint Pierre And Miquelon', 'Saint Vincent And The Grenadines',
       'Sao Tome And Principe', 'South Korea', 'St. Kitts-Nevis-Anguilla',
       'Syria', 'Taiwan', 'Tanzania', 'Timor', 'Trinidad And Tobago',
       'Turks And Caicos Islands',
       'Venezuela', 'Vietnam', 'Wallis And Futuna', 'Ussr',
       'Czechoslovakia', 'Falkland Islands', 'Netherlands Antilles',
       'Serbia And Montenegro', 'United States Virgin Islands',
       'Western Sahara', 'Yugoslavia',
       'Micronesia (Country)', 'Sint Maarten (Dutch Part)'
                  ]
countries = countries_official_names + other_countries

In [44]:
df[x].isna().sum()

0

In [45]:
def assign_category_to_entity(s):

    if "(" in s:
        if "(Bp)" in s:
            return "Bp?"
        if "(Eia)" in s:
            return "Eia?"
        if "(Ember)" in s:
            return "Ember?"
        if s in countries:
            return "Country"
        else:
            return "Exlusive Regions"

    else:
        if s == "World":
            return "World"
        elif s in continents:
            return "Continent"
        elif s in countries:
            return "Country"
        elif "Income" in s:
            return "Income Level"
        else:
            return "Other"

In [46]:
df['Entity_Category'] = df[x].apply(assign_category_to_entity)

In [47]:
df['Entity_Category'].value_counts()

Country      21568
World          272
Continent       39
Name: Entity_Category, dtype: int64

In [57]:
# df.loc[df['Entity_Category'] == 'Continent']

In [49]:
df_entity_lookup = df[['Entity_Category', 'Entity', 'Code', 'Income group', 'Lending category']].drop_duplicates().sort_values(by='Entity_Category')

In [50]:
df_entity_lookup['Entity_Category'].value_counts()

Country      233
Continent      1
World          1
Name: Entity_Category, dtype: int64

In [51]:
df_entity_lookup.head()

,Entity_Category,Entity,Code,Income group,Lending category
835,Continent,Antarctica,ATA,NaN,NaN
0,Country,Aruba,ABW,High income,NaN
13878,Country,New Caledonia,NCL,High income,NaN
13949,Country,Niger,NER,Low income,IDA
14012,Country,Nigeria,NGA,Lower middle income,Blend


In [52]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels per capita (kWh),Oil Consumption - TWh,Economy,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh),Entity_Category
0,Aruba,1926,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33394.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
1,Aruba,1927,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
2,Aruba,1928,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
3,Aruba,1929,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103410.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
4,Aruba,1930,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country


## Save Tables

In [53]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels per capita (kWh),Oil Consumption - TWh,Economy,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh),Entity_Category
0,Aruba,1926,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33394.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
1,Aruba,1927,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
2,Aruba,1928,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
3,Aruba,1929,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103410.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
4,Aruba,1930,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country


In [54]:
df.to_csv('../data/silver_tables/yearly_values_per_entity.csv', index=False)

In [55]:
df_entity_lookup.head()

,Entity_Category,Entity,Code,Income group,Lending category
835,Continent,Antarctica,ATA,NaN,NaN
0,Country,Aruba,ABW,High income,NaN
13878,Country,New Caledonia,NCL,High income,NaN
13949,Country,Niger,NER,Low income,IDA
14012,Country,Nigeria,NGA,Lower middle income,Blend


In [56]:
df_entity_lookup.to_csv('../data/silver_tables/entity_lookup.csv', index=False)